In [14]:
import pandas as pd
import geopandas as gpd
import numpy as np
import requests
import json
from scipy.spatial import cKDTree
import networkx as nx 

In [ ]:
INPUT_FILE = '../data/202501-citibike-tripdata_3.csv'
OUTPUT_FILE = '../data/202501-citibike-sample.csv'
TARGET_ROWS = 100000

print("Reading and sampling...")

df = pd.read_csv(INPUT_FILE)

print(f"Loaded {len(df)} rows.")

if len(df) > TARGET_ROWS:
    df = df.sample(n=TARGET_ROWS)

print(f"Saving {len(df)} rows to {OUTPUT_FILE}...")
df.to_csv(OUTPUT_FILE, index=False)
print("Done! You have a perfect random sample.")

Reading and sampling...
Loaded 124475 rows.
Saving 100000 rows to ../data/202501-citibike-sample.csv...
Done! You have a perfect random sample.


In [ ]:
YEAR = "2022"
DATASET = "acs/acs5"
BASE_URL = f"https://api.census.gov/data/{YEAR}/{DATASET}"
VARIABLES = "NAME,B08201_001E,B08201_002E"

# NYC County Codes (FIPS): Bronx(005), Brooklyn(047), Manhattan(061), Queens(081), Staten Island(085)
counties = ["005", "047", "061", "081", "085"]
state_code = "36" # New York

all_data = []

print("Fetching data for...")
for county in counties:
    url = f"{BASE_URL}?get={VARIABLES}&for=tract:*&in=state:{state_code}&in=county:{county}"
    
    try:
        response = requests.get(url)
        data = response.json()
        headers = data[0]
        rows = data[1:]
        df_county = pd.DataFrame(rows, columns=headers)
        all_data.append(df_county)
        print(f"  - County {county}: Found {len(rows)} tracts")
        
    except Exception as e:
        print(f"  - Error fetching county {county}: {e}")

nyc_df = pd.concat(all_data, ignore_index=True)

nyc_df = nyc_df.rename(columns={
    "B08201_001E": "total_households",
    "B08201_002E": "no_vehicle_households",
    "state": "state_fips",
    "county": "county_fips",
    "tract": "tract_fips"
})

nyc_df["GEOID"] = nyc_df["state_fips"] + nyc_df["county_fips"] + nyc_df["tract_fips"]
nyc_df["total_households"] = pd.to_numeric(nyc_df["total_households"])
nyc_df["no_vehicle_households"] = pd.to_numeric(nyc_df["no_vehicle_households"])
nyc_df["pct_no_vehicle"] = nyc_df["no_vehicle_households"] / nyc_df["total_households"]

nyc_df.to_csv("../data/nyc_transit_equity.csv", index=False)
print("\nSuccess! Saved 'nyc_transit_equity.csv'")

Fetching data for...
  - County 005: Found 361 tracts
  - County 047: Found 805 tracts
  - County 061: Found 310 tracts
  - County 081: Found 725 tracts
  - County 085: Found 126 tracts

Success! Saved 'nyc_transit_equity.csv'


In [ ]:
url = "https://data.cityofnewyork.us/resource/63ge-mke6.geojson?$limit=5000"

print(f"Downloading GeoJSON from {url}...")
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    count = len(data.get('features', []))
    print(f"Success! Downloaded {count} tracts.")
    
    if count > 0:
        with open('../data/nyc_tracts.geojson', 'w') as f:
            json.dump(data, f)
        print("Saved to 'nyc_tracts.geojson'. Move this file to your data/ folder.")
    else:
        print("Error: The downloaded file has 0 features.")
else:
    print(f"Error: Failed to download. Status code: {response.status_code}")

Success! Downloaded 2325 tracts.
Saved to 'nyc_tracts.geojson'. Move this file to your data/ folder.


In [18]:
print("Loading data...")
df_tracts = gpd.read_file('../data/nyc_tracts.geojson')
df_equity = pd.read_csv('../data/nyc_transit_equity.csv')
df_stations = pd.read_csv('../data/202501-citibike-sample.csv')

W_DENSITY = 0.40
W_NEED = 0.45
W_GAP = 0.15

if 'geoid' in df_tracts.columns: df_tracts = df_tracts.rename(columns={'geoid': 'GEOID'})
if 'geoid' in df_equity.columns: df_equity = df_equity.rename(columns={'geoid': 'GEOID'})
df_tracts['GEOID'] = df_tracts['GEOID'].astype(str).str.strip()
df_equity['GEOID'] = df_equity['GEOID'].astype(str).str.strip()

df_stations = df_stations.dropna(subset=['start_lat', 'start_lng'])
station_points = list(zip(df_stations.start_lng, df_stations.start_lat))

df_tracts = df_tracts[df_tracts.geometry.notna()]
df_tracts['centroid'] = df_tracts.geometry.centroid
tract_points = list(zip(df_tracts.centroid.x, df_tracts.centroid.y))

tree = cKDTree(station_points)
dist, idx = tree.query(tract_points, k=1)
df_tracts['dist_to_station'] = dist

df_merged = df_tracts.merge(df_equity, on='GEOID', how='inner')
candidates = df_merged.copy()
candidates = candidates[candidates['dist_to_station'] > 0.005]

for col in candidates.columns:
    if col.lower() in ['boroname', 'boro_name', 'borough']:
        candidates = candidates.rename(columns={col: 'boroname'})
        break

if 'boroname' in candidates.columns:
    print(f"Filtering out Manhattan from {len(candidates)} tracts...")
    candidates = candidates[~candidates['boroname'].isin(['Manhattan', 'New York'])]
    print(f"Remaining tracts: {len(candidates)}")
else:
    print("⚠️ WARNING: Could not find 'boroname' column. Manhattan was not filtered.")

max_density = candidates['total_households'].max()
max_carfree = candidates['pct_no_vehicle'].max()
max_gap = candidates['dist_to_station'].max()

candidates['norm_density'] = candidates['total_households'] / max_density if max_density > 0 else 0
candidates['norm_carfree'] = candidates['pct_no_vehicle'] / max_carfree if max_carfree > 0 else 0
candidates['norm_gap'] = candidates['dist_to_station'] / max_gap if max_gap > 0 else 0

candidates['Expansion_Score'] = (
    (candidates['norm_density'] * W_DENSITY) + 
    (candidates['norm_carfree'] * W_NEED) + 
    (candidates['norm_gap'] * W_GAP)
)

nta_col = 'ntaname' if 'ntaname' in candidates.columns else 'NTAName'
neighborhood_ranks = candidates.groupby(nta_col).agg({
    'Expansion_Score': 'mean'
}).reset_index()

top_candidates = neighborhood_ranks.sort_values('Expansion_Score', ascending=False).head(10)
top_names = top_candidates[nta_col].tolist()
print(f"Top Candidates (Outer Boroughs Only): {top_names}")

all_neighborhoods = df_tracts.dissolve(by=nta_col).reset_index()
candidate_geoms = all_neighborhoods[all_neighborhoods[nta_col].isin(top_names)].copy()

G = nx.Graph()
G.add_nodes_from(top_names)

for i, row1 in candidate_geoms.iterrows():
    for j, row2 in candidate_geoms.iterrows():
        if i >= j: continue 
        if row1.geometry.buffer(0.0001).intersects(row2.geometry):
            G.add_edge(row1[nta_col], row2[nta_col])

mapping = {}
display_groups = []

for component in nx.connected_components(G):
    members = list(component)
    members.sort() 
    group_name = " & ".join(members)
    for member in members:
        mapping[member] = group_name
    display_groups.append(group_name)

output_data = {
    "generated_at": pd.Timestamp.now().strftime("%Y-%m-%d"),
    "target_zones": top_names,
    "group_mapping": mapping,
    "display_list": display_groups
}

with open('../data/gap_data.json', 'w') as f:
    json.dump(output_data, f, indent=2)

print("\n✅ Success! Saved to ../data/gap_data.json")

Loading data...


C:\Users\justi\AppData\Local\Temp\ipykernel_21148\868742574.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_tracts['centroid'] = df_tracts.geometry.centroid
C:\Users\justi\AppData\Local\Temp\ipykernel_21148\868742574.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  tract_points = list(zip(df_tracts.centroid.x, df_tracts.centroid.y))


Filtering out Manhattan from 1234 tracts...
Remaining tracts: 1232
Top Candidates (Outer Boroughs Only): ['Spring Creek-Starrett City', 'Parkchester', 'Brighton Beach', 'Flushing-Willets Point', 'Co-op City', 'Coney Island-Sea Gate', 'Norwood', 'Soundview-Bruckner-Bronx River', 'Rockaway Beach-Arverne-Edgemere', 'Brownsville']

✅ Success! Saved to ../data/gap_data.json
